# 4. LangChain Memory - Gestión de Contexto Conversacional

## Objetivos de Aprendizaje
- Comprender la importancia de la memoria en conversaciones con LLMs
- Implementar diferentes tipos de memoria con LangChain
- Gestionar el contexto de conversaciones largas
- Optimizar el uso de tokens con estrategias de memoria

## ¿Por qué es Importante la Memoria?

Los LLMs son **stateless** por naturaleza: no recuerdan conversaciones anteriores. La memoria permite:
- **Contexto conversacional**: Referirse a mensajes anteriores
- **Personalización**: Recordar preferencias del usuario
- **Continuidad**: Mantener hilos de conversación coherentes
- **Experiencia natural**: Conversaciones que se sienten humanas

## Tipos de Memoria en LangChain

1. **ConversationBufferMemory**: Mantiene todo el historial
2. **ConversationSummaryMemory**: Resume conversaciones largas
3. **ConversationBufferWindowMemory**: Mantiene solo los N mensajes más recientes
4. **ConversationSummaryBufferMemory**: Combina resumen + buffer reciente

In [ ]:
# Importar bibliotecas necesarias para memoria
from langchain_openai import ChatOpenAI
from langchain.memory import (
    ConversationBufferMemory,
    ConversationSummaryMemory,
    ConversationBufferWindowMemory
)
from langchain.chains import ConversationChain
import os

print("✓ Bibliotecas de memoria importadas correctamente")

In [ ]:
# Configuración del modelo para memoria
try:
    """ llm = ChatOpenAI(
        base_url=os.getenv("OPENAI_BASE_URL"),
        api_key=os.getenv("GITHUB_TOKEN"),
        model="gpt-4o",
        temperature=0.7
    ) """
    
    llm = ChatOpenAI(
        base_url="https://models.github.ai/inference",  #descubri que aqui  cambia de url base  
        api_key=os.getenv("GITHUB_TOKEN"),               
        model = "openai/gpt-4.1",               # modelo DeepSeek
        temperature=0.7,
        # max_tokens=150
    )
    
    print("✓ Modelo configurado para experimentos de memoria")
    print(f"Modelo: {llm.model_name}")
    
except Exception as e:
    print(f"✗ Error en configuración: {e}")
    print("Verifica las variables de entorno")

## 1. ConversationBufferMemory - Memoria Completa

Esta memoria mantiene **todo** el historial de la conversación. Es la más simple pero puede consumir muchos tokens.

In [6]:
# Ejemplo básico con ConversationBufferMemory
def ejemplo_buffer_memory():
    print("=== CONVERSATIONBUFFERMEMORY ===")
    print("Mantiene todo el historial de conversación\\n")
    
    try:
        # Crear memoria buffer
        memory = ConversationBufferMemory()
        
        # Crear cadena de conversación
        conversation = ConversationChain(
            llm=llm,
            memory=memory,
            verbose=True  # Muestra el prompt interno
        )
        
        # Primera interacción
        print("1. Primera pregunta:")
        response1 = conversation.predict(input="Mi nombre es Pedro y soy programadora C#")
        print(f"Respuesta: {response1}\\n")
        
        # Segunda interacción (debe recordar el nombre)
        print("2. Segunda pregunta:")
        response2 = conversation.predict(input="¿Cuál es mi nombre y profesión?")
        print(f"Respuesta: {response2}\\n")
        
        # Tercera interacción (debe recordar todo el contexto)
        print("3. Tercera pregunta:")
        response3 = conversation.predict(input="¿Qué lenguaje de programación mencioné?")
        print(f"Respuesta: {response3}\\n")
        
        # Examinar el contenido de la memoria
        print("=== CONTENIDO DE LA MEMORIA ===")
        print(memory.buffer)
        
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar ejemplo
ejemplo_buffer_memory()

=== CONVERSATIONBUFFERMEMORY ===
Mantiene todo el historial de conversación\n
1. Primera pregunta:


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Mi nombre es Pedro y soy programadora C#
AI:

> Finished chain.
Respuesta: ¡Hola, Pedro! Es un gusto conocerte. Qué chévere que seas programadora en C#. ¿En qué tipo de proyectos has trabajado? C# es muy versátil: desde aplicaciones de escritorio con Windows Forms o WPF, hasta desarrollo web con ASP.NET Core, y hasta juegos con Unity. Si tienes alguna pregunta sobre C# o quieres compartir algún reto que hayas tenido, ¡cuéntame!\n
2. Segunda pregunta:


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation betwe

## 2. ConversationBufferWindowMemory - Ventana Deslizante

Esta memoria mantiene solo los **N mensajes más recientes**, útil para controlar el uso de tokens.

In [7]:
# Ejemplo con ConversationSummaryMemory
def ejemplo_summary_memory():
    print("=== CONVERSATIONSUMMARYMEMORY ===")
    print("Resume conversaciones largas para ahorrar tokens\\n")
    
    try:
        # Crear memoria con resumen
        memory = ConversationSummaryMemory(llm=llm)
        
        conversation = ConversationChain(
            llm=llm,
            memory=memory,
            verbose=True
        )
        
        # Simular una conversación larga con muchos detalles
        long_inputs = [
            "Hola, me llamo María González, tengo 35 años y soy ingeniera de software especializada en desarrollo web con React y Node.js. Trabajo en una startup de fintech en Madrid desde hace 3 años.",
            "Mi proyecto actual involucra crear una plataforma de pagos digitales que debe manejar transacciones en tiempo real con alta seguridad. Usamos microservicios con Docker y Kubernetes.",
            "El mayor desafío técnico que enfrentamos es la latencia en las transacciones internacionales. Estamos considerando implementar edge computing y optimizar nuestras APIs.",
            "También estoy trabajando en mejorar la experiencia de usuario de nuestra aplicación móvil. Los usuarios se quejan de que el proceso de verificación de identidad es muy lento.",
            "¿Puedes resumir quién soy y cuáles son mis principales desafíos profesionales?"
        ]
        
        for i, user_input in enumerate(long_inputs, 1):
            print(f"{i}. Input: {user_input[:100]}...")
            response = conversation.predict(input=user_input)
            print(f"   Respuesta: {response}\\n")
            
            # Mostrar el resumen actual
            print("   Resumen actual en memoria:")
            print(f"   {memory.buffer}\\n")
            print("-" * 80)
            
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar ejemplo
ejemplo_summary_memory()

=== CONVERSATIONSUMMARYMEMORY ===
Resume conversaciones largas para ahorrar tokens\n
1. Input: Hola, me llamo María González, tengo 35 años y soy ingeniera de software especializada en desarrollo...


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola, me llamo María González, tengo 35 años y soy ingeniera de software especializada en desarrollo web con React y Node.js. Trabajo en una startup de fintech en Madrid desde hace 3 años.
AI:


C:\Users\Alejandro\AppData\Local\Temp\ipykernel_4364\1771842198.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)



> Finished chain.
   Respuesta: ¡Hola, María! Encantada de conocerte. Qué interesante tu perfil profesional: React y Node.js son tecnologías súper demandadas hoy en día, especialmente en el sector fintech, donde la agilidad y la escalabilidad son clave. Madrid tiene una comunidad tech muy activa, y las startups fintech están revolucionando el mundo financiero con soluciones innovadoras.

Si te apetece, cuéntame más sobre tu trabajo: ¿en qué tipo de proyectos te sueles involucrar? ¿Utilizáis otras tecnologías en tu stack, como TypeScript, GraphQL o herramientas de CI/CD? También, si tienes alguna pregunta sobre desarrollo web, tendencias en fintech, o incluso sobre cómo mejorar procesos dentro de tu equipo, estaré encantada de ayudarte.\n
   Resumen actual en memoria:
   María González, una ingeniera de software especializada en desarrollo web con React y Node.js, comparte que trabaja en una startup fintech en Madrid desde hace 3 años. El AI destaca el valor de su perfil profesional y 

## 3. ConversationSummaryMemory - Resumen Inteligente

Esta memoria **resume** conversaciones largas en lugar de mantener todo el texto completo, ahorrando tokens significativamente.

In [ ]:
# Ejemplo con ConversationBufferWindowMemory
def ejemplo_window_memory():
    print("=== CONVERSATIONBUFFERWINDOWMEMORY ===")
    print("Mantiene solo los 2 intercambios más recientes\\n")
    
    try:
        # Crear memoria con ventana de 2 intercambios
        memory = ConversationBufferWindowMemory(k=2)
        
        conversation = ConversationChain(
            llm=llm,
            memory=memory,
            verbose=True
        )
        
        # Múltiples interacciones para ver el efecto de la ventana
        inputs = [
            "Mi nombre es Carlos y tengo 30 años",
            "Trabajo como diseñador gráfico",
            "Me gusta el café y la música jazz",
            "¿Puedes recordar mi edad?",  # Debería olvidar esto
            "¿Cuál es mi profesión?"  # Debería recordar esto
        ]
        
        for i, user_input in enumerate(inputs, 1):
            print(f"{i}. Pregunta: {user_input}")
            response = conversation.predict(input=user_input)
            print(f"   Respuesta: {response}\\n")
            
            # Mostrar contenido actual de la memoria
            print(f"   Memoria actual (k=2):")
            print(f"   {memory.buffer}\\n")
            print("-" * 60)
            
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar ejemplo
ejemplo_window_memory()

## Comparación de Tipos de Memoria

Veamos las diferencias entre los tipos de memoria en una misma conversación.

In [ ]:
# Comparación entre tipos de memoria
def comparar_memorias():
    print("=== COMPARACIÓN DE TIPOS DE MEMORIA ===\\n")
    
    # Configurar diferentes tipos de memoria
    buffer_memory = ConversationBufferMemory()
    window_memory = ConversationBufferWindowMemory(k=2)
    summary_memory = ConversationSummaryMemory(llm=llm)
    
    # Crear conversaciones con cada tipo
    conversations = {
        "Buffer (Todo)": ConversationChain(llm=llm, memory=buffer_memory, verbose=False),
        "Window (k=2)": ConversationChain(llm=llm, memory=window_memory, verbose=False),
        "Summary": ConversationChain(llm=llm, memory=summary_memory, verbose=False)
    }
    
    # Secuencia de inputs para probar
    test_inputs = [
        "Mi nombre es Alex y estudio ingeniería informática",
        "Tengo 22 años y me especializo en IA",
        "Mi lenguaje favorito es Python",
        "También me gusta JavaScript para desarrollo web",
        "¿Cuál es mi edad y carrera?"  # Pregunta que requiere memoria
    ]
    
    # Ejecutar la misma conversación con cada tipo de memoria
    for memory_type, conversation in conversations.items():
        print(f"\\n{'='*20} {memory_type.upper()} {'='*20}")
        
        for i, user_input in enumerate(test_inputs, 1):
            try:
                if i < len(test_inputs):  # No mostrar la pregunta final aún
                    response = conversation.predict(input=user_input)
                    print(f"{i}. Input: {user_input}")
                    print(f"   Respuesta: {response[:100]}...")
                else:  # Pregunta final para probar memoria
                    response = conversation.predict(input=user_input)
                    print(f"\\n{i}. PREGUNTA DE MEMORIA: {user_input}")
                    print(f"   RESPUESTA: {response}")
                    
                    # Mostrar estado de memoria
                    if memory_type == "Buffer (Todo)":
                        print(f"   Memoria: {len(buffer_memory.buffer)} caracteres")
                    elif memory_type == "Window (k=2)":
                        print(f"   Memoria: {len(window_memory.buffer)} caracteres")
                    else:
                        print(f"   Resumen: {len(summary_memory.buffer)} caracteres")
                        
            except Exception as e:
                print(f"   Error: {e}")
        
        print("\\n" + "-"*60)
    
    print("\\n=== ANÁLISIS ===")
    print("• Buffer Memory: Recuerda todo pero consume más tokens")
    print("• Window Memory: Eficiente pero puede olvidar información importante")  
    print("• Summary Memory: Balance entre memoria y eficiencia")

# Ejecutar comparación
comparar_memorias()

## Chatbot Avanzado con Memoria Personalizable

Implementemos un chatbot que permite al usuario elegir el tipo de memoria.

In [ ]:
# Chatbot avanzado con memoria configurable
def chatbot_con_memoria():
    print("=== CHATBOT CON MEMORIA CONFIGURABLE ===")
    print("Tipos disponibles:")
    print("1. buffer - Mantiene todo el historial")
    print("2. window - Mantiene solo N mensajes recientes") 
    print("3. summary - Resume conversaciones largas")
    print("\\nEscribe 'cambiar' para cambiar tipo de memoria")
    print("Escribe 'memoria' para ver el contenido actual")
    print("Escribe 'salir' para terminar\\n")
    
    # Configuración inicial
    memory_type = "buffer"
    memory = ConversationBufferMemory()
    conversation = ConversationChain(llm=llm, memory=memory, verbose=False)
    
    def crear_memoria(tipo):
        if tipo == "buffer":
            return ConversationBufferMemory()
        elif tipo == "window":
            return ConversationBufferWindowMemory(k=3)
        elif tipo == "summary":
            return ConversationSummaryMemory(llm=llm)
        else:
            return ConversationBufferMemory()
    
    print(f"Memoria actual: {memory_type}")
    
    while True:
        user_input = input("\\n🧑 Tú: ")
        
        if user_input.lower() == 'salir':
            print("\\n👋 ¡Hasta luego!")
            break
            
        elif user_input.lower() == 'cambiar':
            print("\\nTipos disponibles: buffer, window, summary")
            nuevo_tipo = input("Nuevo tipo de memoria: ").lower()
            
            if nuevo_tipo in ['buffer', 'window', 'summary']:
                memory_type = nuevo_tipo
                memory = crear_memoria(memory_type)
                conversation = ConversationChain(llm=llm, memory=memory, verbose=False)
                print(f"✓ Memoria cambiada a: {memory_type}")
                print("⚠️ Historial de conversación reiniciado")
            else:
                print("❌ Tipo no válido")
            continue
            
        elif user_input.lower() == 'memoria':
            print(f"\\n=== MEMORIA ACTUAL ({memory_type}) ===")
            print(f"Contenido: {memory.buffer}")
            print(f"Longitud: {len(memory.buffer)} caracteres")
            continue
            
        elif not user_input.strip():
            continue
        
        try:
            print(f"\\n🤖 Asistente ({memory_type}): ", end="", flush=True)
            response = conversation.predict(input=user_input)
            print(response)
            
        except Exception as e:
            print(f"❌ Error: {e}")

# Función para ejecutar el chatbot
# chatbot_con_memoria()  # Descomenta para ejecutar

print("💡 Descomenta la línea anterior para probar el chatbot con memoria configurable")

## Consideraciones Técnicas y Mejores Prácticas

### Selección del Tipo de Memoria

| Tipo | Cuándo Usarlo | Ventajas | Desventajas |
|------|---------------|----------|-------------|
| **Buffer** | Conversaciones cortas | Contexto completo | Alto consumo de tokens |
| **Window** | Contexto reciente importante | Eficiente en tokens | Puede perder información clave |
| **Summary** | Conversaciones muy largas | Balance eficiencia/contexto | Pérdida de detalles específicos |

### Mejores Prácticas:

1. **Gestión de Tokens**:
   - Monitorea el uso de tokens regularmente
   - Establece límites máximos para evitar costos excesivos
   - Considera el costo vs. calidad del contexto

2. **Selección Estratégica**:
   - Usa Buffer para sesiones cortas e importantes
   - Usa Window para conversaciones con contexto limitado
   - Usa Summary para sesiones largas de asistencia

3. **Optimización**:
   - Limpia memoria periódicamente si es necesario
   - Implementa estrategias híbridas según el caso de uso
   - Considera almacenamiento persistente para memoria a largo plazo

## Ejercicios Prácticos

### Ejercicio 1: Análisis de Consumo
Implementa un sistema que monitoree y reporte el uso de tokens con diferentes tipos de memoria.

### Ejercicio 2: Memoria Híbrida
Diseña una estrategia que combine multiple tipos de memoria según el contexto.

### Ejercicio 3: Persistencia
Extiende el chatbot para guardar y cargar memoria entre sesiones.

## Conceptos Clave Aprendidos

1. **Importancia de la memoria** en conversaciones naturales
2. **Tipos de memoria** y sus casos de uso específicos
3. **Balance** entre contexto y eficiencia de tokens
4. **Implementación práctica** con LangChain
5. **Estrategias de optimización** para diferentes escenarios

## Conclusión del Módulo IL1.1

Has completado la introducción a LLMs y conexiones API. Los conceptos aprendidos:

1. **APIs directas** vs **frameworks** como LangChain
2. **Streaming** para mejor experiencia de usuario
3. **Memoria** para conversaciones contextuales
4. **Mejores prácticas** de seguridad y optimización

### Próximos Pasos
En **IL1.2** exploraremos técnicas avanzadas de **prompt engineering** incluyendo zero-shot, few-shot, y chain-of-thought prompting.